# 01 – Preprocessing (City-level EV Dataset)

Objective: Ingest raw EV population Excel, clean, and produce city-level aggregates suitable for clustering and modeling.

Outputs:
- `data/processed/city_ev_agg.csv` – City-level counts per year and total.
- `data/processed/model_ready_city_ev.csv` – Per city/year with lag features.

Requirements: pandas, numpy.


In [ ]:
# Imports: data wrangling and file path utilities
import pandas as pd
import numpy as np
from pathlib import Path


In [ ]:
# Resolve paths robustly whether run from project root or notebooks/
CWD = Path.cwd()
# Decide base directory (project root). If running inside notebooks/, go one level up.
if CWD.name == 'notebooks':
    BASE_DIR = CWD.parent
else:
    BASE_DIR = CWD

# Candidate locations for the raw Excel; pick the first that exists
CANDIDATES = [
    BASE_DIR / 'data/raw/Electric_Vehicle_Population_Data.xlsx',
    CWD / 'data/raw/Electric_Vehicle_Population_Data.xlsx',  # fallback when running from notebooks/
]
RAW_PATH = next((p for p in CANDIDATES if p.exists()), None)
if RAW_PATH is None:
    raise FileNotFoundError(f"Could not find raw Excel. Tried: {[str(p) for p in CANDIDATES]}")

# Ensure output directory exists
OUT_DIR = BASE_DIR / 'data/processed'
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Diagnostics to help users see what paths are being used
print(f"Current directory: {CWD}")
print(f"Base directory: {BASE_DIR}")
print(f"Raw data path: {RAW_PATH}")
print(f"Output directory: {OUT_DIR}")
print(f"Raw data exists: {RAW_PATH.exists()}")
print(f"Output directory exists: {OUT_DIR.exists()}")


Raw data path: e:\FDM\PROJECT\NEW\Smart-Charge-Locator\data\raw\Electric_Vehicle_Population_Data.xlsx
Output directory: e:\FDM\PROJECT\NEW\Smart-Charge-Locator\notebooks\data\processed


In [ ]:
# Read raw Excel (first sheet)
# Note: requires openpyxl in the environment
df = pd.read_excel(RAW_PATH)
print(f"Raw data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")


Raw data shape: (250659, 17)
Columns: ['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year', 'Make', 'Model', 'Electric Vehicle Type', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range', 'Base MSRP', 'Legislative District', 'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility', '2020 Census Tract']


In [ ]:
# Standardize column names (trim spaces, Title Case)
df.columns = [c.strip().title() for c in df.columns]

# Expected columns (best-effort); warn if any are missing
expected = ['City', 'County', 'Postal Code', 'Model Year', 'Electric Range']
missing = [c for c in expected if c not in df.columns]
if missing:
    print('Warning: missing columns:', missing)

# Keep only relevant columns that exist, and optional geo columns if available
extra_cols = [c for c in ['Vehicle Location','Latitude','Longitude'] if c in df.columns]
df = df[[c for c in expected if c in df.columns] + extra_cols]

print(f"After column selection: {df.shape}")
print(f"Selected columns: {list(df.columns)}")


After column selection: (250659, 6)
Selected columns: ['City', 'County', 'Postal Code', 'Model Year', 'Electric Range', 'Vehicle Location']


In [ ]:
# Coerce types for consistency and downstream math

# Ensure model year is numeric (coerce invalid values to NaN)
if 'Model Year' in df:
    df['Model Year'] = pd.to_numeric(df['Model Year'], errors='coerce')
# Ensure postal code is numeric if present (useful for nunique)
if 'Postal Code' in df:
    df['Postal Code'] = pd.to_numeric(df['Postal Code'], errors='coerce')

# Drop rows without City and normalize casing/whitespace for grouping
if 'City' in df:
    df = df.dropna(subset=['City'])
    df['City'] = df['City'].astype(str).str.strip().str.title()

print(f"After cleaning: {df.shape}")
print(f"City column sample: {df['City'].head()}")


After cleaning: (250653, 6)
City column sample: 0      Granger
1       Yakima
2      Everett
3       Yakima
4    Bremerton
Name: City, dtype: object


In [ ]:
# Build city-year counts (each row in raw is one vehicle registration)
city_year = df.groupby(['City','Model Year'], as_index=False).size().rename(columns={'size':'EV_Count'})

print(f"City-year data shape: {city_year.shape}")
print("Sample city-year data:")
city_year.head()


City-year data shape: (5190, 3)
Sample city-year data:


,City,Model Year,EV_Count
0,Aberdeen,2011,1
1,Aberdeen,2013,8
2,Aberdeen,2014,4
3,Aberdeen,2015,5
4,Aberdeen,2016,7


In [ ]:
# Create lag features per city(feature engineering)
# - Prev_Year_EV_Count: previous year's count, fill 0 for first year
# - Year_Delta: how many years since the earliest model year in dataset
city_year = city_year.sort_values(['City','Model Year'])
city_year['Prev_Year_EV_Count'] = city_year.groupby('City')['EV_Count'].shift(1).fillna(0)
city_year['Year_Delta'] = city_year['Model Year'] - city_year['Model Year'].min()

print("Features created:")
print("- Prev_Year_EV_Count: Previous year's EV count for each city")
print("- Year_Delta: Years since the earliest model year")
print("\nSample with features:")
city_year.head()


Features created:
- Prev_Year_EV_Count: Previous year's EV count for each city
- Year_Delta: Years since the earliest model year

Sample with features:


,City,Model Year,EV_Count,Prev_Year_EV_Count,Year_Delta
0,Aberdeen,2011,1,0.0,11
1,Aberdeen,2013,8,1.0,13
2,Aberdeen,2014,4,8.0,14
3,Aberdeen,2015,5,4.0,15
4,Aberdeen,2016,7,5.0,16


In [ ]:
# Aggregate city totals for clustering baseline and reporting
city_totals = city_year.groupby('City', as_index=False)['EV_Count'].sum().rename(columns={'EV_Count':'EV_Count_Total'})

print(f"City totals shape: {city_totals.shape}")
print("Sample city totals:")
city_totals.head()


City totals shape: (821, 2)
Sample city totals:


,City,EV_Count_Total
0,Aberdeen,236
1,Aberdeen Proving Ground,1
2,Acme,11
3,Addy,6
4,Airway Heights,42


In [ ]:
# Density proxy: EVs per unique postal code in the city
# Rationale: quick proxy when population/area not available
if 'Postal Code' in df:
    zips_per_city = df.groupby('City', as_index=False)['Postal Code'].nunique().rename(columns={'Postal Code':'Unique_Zips'})
    city_totals = city_totals.merge(zips_per_city, on='City', how='left')
    city_totals['Unique_Zips'] = city_totals['Unique_Zips'].replace(0, np.nan)
    city_totals['EV_Density_Proxy'] = city_totals['EV_Count_Total'] / city_totals['Unique_Zips']
else:
    city_totals['EV_Density_Proxy'] = city_totals['EV_Count_Total']

print("EV Density Proxy calculation:")
print("- Formula: Total EVs / Unique Postal Codes in city")
print("- Purpose: Proxy for EV density when population/area data unavailable")
print("\nSample with density proxy:")
city_totals.head()


EV Density Proxy calculation:
- Formula: Total EVs / Unique Postal Codes in city
- Purpose: Proxy for EV density when population/area data unavailable

Sample with density proxy:


,City,EV_Count_Total,Unique_Zips,EV_Density_Proxy
0,Aberdeen,236,1,236.0
1,Aberdeen Proving Ground,1,1,1.0
2,Acme,11,1,11.0
3,Addy,6,1,6.0
4,Airway Heights,42,1,42.0


In [ ]:
# Save artifacts for downstream steps (clustering and modeling)
city_totals.to_csv(OUT_DIR / 'city_ev_agg.csv', index=False)
city_year.to_csv(OUT_DIR / 'model_ready_city_ev.csv', index=False)

print("Files saved:")
print(f"- {OUT_DIR / 'city_ev_agg.csv'}")
print(f"- {OUT_DIR / 'model_ready_city_ev.csv'}")
print("\nFinal summary:")
print(f"Total cities: {len(city_totals)}")
print(f"Total city-year records: {len(city_year)}")
print(f"Year range: {city_year['Model Year'].min()} - {city_year['Model Year'].max()}")


Files saved:
- e:\FDM\PROJECT\NEW\Smart-Charge-Locator\notebooks\data\processed\city_ev_agg.csv
- e:\FDM\PROJECT\NEW\Smart-Charge-Locator\notebooks\data\processed\model_ready_city_ev.csv

Final summary:
Total cities: 821
Total city-year records: 5190
Year range: 2000 - 2026
